In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_colwidth=300

np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.2f}'.format


pd.set_option('display.max_columns', None)

# Read data
df_train = pd.read_csv("Data/instagram_data.csv", index_col=0)

df_train.reset_index(inplace=True)
df_train.rename(columns={'index': 'No.'}, inplace=True)

display(df_train.head(), df_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           845 non-null    object 
 1   name              826 non-null    object 
 2   posts             847 non-null    object 
 3   followers         845 non-null    object 
 4   following         829 non-null    object 
 5   PVT               850 non-null    bool   
 6   date_joined       847 non-null    object 
 7   former_usernames  343 non-null    float64
dtypes: bool(1), float64(1), object(6)
memory usage: 47.4+ KB


,user_id,name,posts,followers,following,PVT,date_joined,former_usernames
0,abagael.taylor,Abagael💐,42,"1,813","1,540",False,January 2019,1.00
1,abagael___,abby,1,4,4,False,May 2015,NaN
2,apbagael,Abagael,8,46,207,False,July 2024,1.00
3,knabagael,Abagael,20,87,408,False,October 2023,1.00
4,artagael,Abagael West,20,112,291,False,August 2017,NaN


None

In [4]:
# Dataset Description
data_dict = pd.read_csv('Data/instagram_data.csv', index_col = False)
data_dict

,user_id,name,posts,followers,following,PVT,date_joined,former_usernames
0,abagael.taylor,Abagael💐,42,"1,813","1,540",False,January 2019,1.00
1,abagael___,abby,1,4,4,False,May 2015,NaN
2,apbagael,Abagael,8,46,207,False,July 2024,1.00
3,knabagael,Abagael,20,87,408,False,October 2023,1.00
4,artagael,Abagael West,20,112,291,False,August 2017,NaN
...,...,...,...,...,...,...,...,...
845,juliaanalvarez,JULIÁN ALVAREZ🕷,502,16.9M,644,False,January 2014,NaN
846,danielaalvareztv,Daniela Alvarez,"3,630",4.7M,"4,670",False,November 2012,NaN
847,criseldalvarez,criselda alvarez,83,1.8M,1,False,March 2015,NaN
848,ealvarezfight,Eddie Alvarez,"1,381",389K,"1,947",False,May 2013,NaN


In [5]:
num_cols = ['ratio_numlen_username','len_fullname','ratio_numlen_fullname',
            'len_desc','num_posts','num_followers',
              'num_following']
cat_cols = [col for col in df_train.columns.values.tolist() if col not in num_cols]
cat_cols

['user_id',
 'name',
 'posts',
 'followers',
 'following',
 'PVT',
 'date_joined',
 'former_usernames']

In [6]:
df_train[num_cols].describe()

KeyError: "None of [Index(['ratio_numlen_username', 'len_fullname', 'ratio_numlen_fullname',\n       'len_desc', 'num_posts', 'num_followers', 'num_following'],\n      dtype='object')] are in the [columns]"

In [ ]:
plt.figure(figsize=(15,6))
fake_share = df_train["fake"].value_counts()
mylabel=["Not fake(0)","fake(1)"]
colors = ['#99ff99','#ff9999']
plt.pie(fake_share,
        labels=mylabel,autopct="%1.1f%%",colors=colors,
        textprops={'fontsize': 16})
plt.axis("equal");

In [ ]:
percent_missing = df_train.isnull().sum() * 100 / len(df_train)
missing_value_df = pd.DataFrame({'percent_missing (%)': percent_missing})
missing_value_df.sort_values('percent_missing (%)', ascending=False)


In [ ]:
corr = df_train[num_cols].corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
pp_cols = []
pp_cols = num_cols + ['fake']
ax=sns.pairplot(df_train[pp_cols], hue="fake",corner=True);
plt.style.use('fivethirtyeight')
ax.fig.suptitle("Pair Plot of Characteristics")

In [ ]:
display(df_train.loc[:, cat_cols].head())
print('----------------------')

for col in cat_cols:
    unique_values = df_train.loc[:, col].unique()
    print("\nColumn name: {}\nUnique values: {}".format(col, unique_values))

In [ ]:
def label_encoding(df):
    dict_label_encoding = {'Yes': 1, 'No': 0}
    df.loc[:, 'profile_pic'] = df.loc[:, 'profile_pic'].replace(dict_label_encoding)
    df.loc[:, 'extern_url'] = df.loc[:, 'extern_url'].replace(dict_label_encoding)
    df.loc[:, 'private'] = df.loc[:, 'private'].replace(dict_label_encoding)

    onehot = pdp.OneHotEncode(["sim_name_username"], drop_first=False)
    df = onehot.fit_transform(df)
    return df

df_train = label_encoding(df_train)
df_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
features_train = df_train.iloc[:, 1:]
target_train = df_train.loc[:, 'fake']

model_log = LogisticRegression(solver='lbfgs', max_iter=1e4, C=1e42, random_state=42)

model_log.fit(features_train, target_train)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)

model_reg = LogisticRegression(solver='lbfgs', max_iter=1e4, C=0.5, random_state=42)

model_reg.fit(features_train_scaled, target_train)